In [1]:
import numpy as np
from PIL import Image


In [2]:
w, h = 300,300
image_array = np.zeros((h,w,4), dtype=np.uint8)

In [3]:
s_s=100
h_s = s_s //2
c_x, c_y =w//2, h//2

image_array[c_y-h_s:c_y+h_s, c_x-h_s:c_x+h_s]= [255,0,0,255]
image = Image.fromarray(image_array,'RGBA')
image.save('rgba.png')
image.show()

In [4]:
import numpy as np
from PIL import Image

# Define image dimensions and number of channels
width, height, num_channels = 300, 300, 6

# Create an image array with the specified number of channels
image_array = np.zeros((height, width, num_channels), dtype=np.uint8)

# Define some example data for the channels
# For simplicity, we'll use gradient data for each channel
for i in range(num_channels):
    image_array[:, :, i] = (i * 255 // num_channels)

# Convert the numpy array to an image (PIL supports up to 4 channels directly)
# To handle more than 4 channels, you might need to save it as a raw file or use specialized libraries
image = Image.fromarray(image_array[:, :, :4], 'RGBA')  # Using the first 4 channels

# Save or display the image
image.save('multi_channel_image.png')
image.show()


In [5]:
import torch


In [6]:
x = torch.tensor([1,2,3,4])

In [7]:
x.shape

torch.Size([4])

In [16]:
torch.unsqueeze(x, 1)

tensor([[1],
        [2],
        [3],
        [4]])

In [14]:
x.shape

torch.Size([4])

In [19]:
x.unsqueeze(1)

tensor([[1],
        [2],
        [3],
        [4]])

In [52]:
import torch
import torch.nn as nn

# Parameters
vocab_size = 3  # Example vocabulary size
d_model = 5 # Dimension of the embedding vectors

# Create an embedding layer
embedding = nn.Embedding(vocab_size, d_model)

# Example token indices (batch of sequences)
token_indices = torch.tensor([0,1,2])

# Get embeddings
embedded_tokens = embedding(token_indices)

print("Token indices:", token_indices)
print("Embedded tokens:", embedded_tokens)


Token indices: tensor([0, 1, 2])
Embedded tokens: tensor([[-1.4959, -1.3505, -1.9001,  1.9507,  0.2111],
        [-0.4325, -1.0526, -0.6222, -0.6579,  0.3872],
        [ 0.8317, -0.5908, -0.6884, -0.8178, -0.3337]],
       grad_fn=<EmbeddingBackward0>)


In [94]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        positions = torch.arange(0, max_len).unsqueeze(1).float()
        #print('hi', positions)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        #print('dvi', div_term)
        self.encoding[:, 0::2] = torch.sin(positions * div_term)
        self.encoding[:, 1::2] = torch.cos(positions * div_term)
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        print(" to be added", self.encoding[:, :x.size(1)])
        return x + self.encoding[:, :x.size(1)]

# Instantiate PositionalEncoding
d_model = 4 # embedding dimension
max_len = 4 # max seq length
pos_encoding = PositionalEncoding(d_model, max_len)
#print(pos_encoding.encoding)
# Generate dummy input embeddings
batch_size = 3
seq_len = 2
input_embeddings = torch.rand(batch_size, seq_len, d_model)
print("Input Embeddings:")
print(input_embeddings)

# Apply positional encoding
encoded_input = pos_encoding(input_embeddings)
print("\nEncoded Input:")
print(encoded_input)



Input Embeddings:
tensor([[[0.6903, 0.5204, 0.8338, 0.3914],
         [0.1914, 0.5398, 0.6105, 0.4342]],

        [[0.6648, 0.7955, 0.1776, 0.1077],
         [0.4159, 0.0324, 0.9512, 0.1627]],

        [[0.3175, 0.7245, 0.7347, 0.0975],
         [0.7690, 0.5583, 0.3212, 0.8656]]])
 to be added tensor([[[0.0000, 1.0000, 0.0000, 1.0000],
         [0.8415, 0.5403, 0.0100, 0.9999]]])

Encoded Input:
tensor([[[0.6903, 1.5204, 0.8338, 1.3914],
         [1.0329, 1.0801, 0.6205, 1.4342]],

        [[0.6648, 1.7955, 0.1776, 1.1077],
         [1.2574, 0.5727, 0.9612, 1.1626]],

        [[0.3175, 1.7245, 0.7347, 1.0975],
         [1.6105, 1.0986, 0.3312, 1.8655]]])


In [123]:
import torch
import torch.nn as nn
import math

class SelfAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super(SelfAttention, self).__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        assert d_model % n_heads == 0
        self.d_k = d_model // n_heads

        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.out_linear = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, mask=None):
        batch_size = q.size(0)
        q = self.q_linear(q)
        q = self.q_linear(q).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        k = self.k_linear(k).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        v = self.v_linear(v).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)

        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        attn = torch.nn.functional.softmax(scores, dim=-1)
        output = torch.matmul(attn, v)
        print("out", output.shape)
        output = output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        return self.out_linear(output)


In [124]:
import torch
import torch.nn as nn
import math
# Define model dimensions and the number of attention heads
d_model = 6  # Dimension of the model (each token embedding)
n_heads = 3  # Number of attention heads

# Create an instance of the SelfAttention class
self_attention = SelfAttention(d_model, n_heads)
 
# Batch size: 2, Sequence length: 5, Embedding dimension: 64 (same as d_model)
batch_size = 2
seq_len = 5

# Randomly generated tensor representing token embeddings (e.g., from an embedding layer)
# This could be output from an embedding layer or another neural network layer.
q = torch.rand(batch_size, seq_len, d_model)  # Query tensor
k = torch.rand(batch_size, seq_len, d_model)  # Key tensor (typically same as q in self-attention)
v = torch.rand(batch_size, seq_len, d_model)  # Value tensor (typically same as q in self-attention)
# Batch size: 2, Sequence length: 5, Embedding dimension: 64 (same as d_model)
batch_size = 2
seq_len = 5

# Randomly generated tensor representing token embeddings (e.g., from an embedding layer)
# This could be output from an embedding layer or another neural network layer.
q = torch.rand(batch_size, seq_len, d_model)  # Query tensor
k = torch.rand(batch_size, seq_len, d_model)  # Key tensor (typically same as q in self-attention)
v = torch.rand(batch_size, seq_len, d_model)  # Value tensor (typically same as q in self-attention)
# Pass the query, key, and value tensors through the self-attention layer
output = self_attention(q, k, v)

# Output tensor shape: (batch_size, seq_len, d_model)
print(output.shape)  # Should print: torch.Size([2, 5, 64])


out torch.Size([2, 3, 5, 2])
torch.Size([2, 5, 6])
